In [12]:
# Code to create the dataset
import pandas as pd

data = {
    'Movie_Name': [
        'The Wizard’s Quest', 'Starlight Chronicles', 'The Dark Sorcerer', 'Moonlit Shadows',
        'Galactic Pioneers', 'Time Loop', 'The Magic Academy', 'Desert Nomads',
        'Cyber Rebellion', 'The Lost Kingdom', 'Echoes of Eternity', 'Skyward Bound',
        'The Phantom Thief', 'Ocean’s Secret', 'The Fire Within', 'Quantum Voyage',
        'The Enchanted Forest', 'Nightmare’s End', 'The Last Guardian', 'Cosmic Dawn'
    ],
    'Storyline': [
        'A young wizard uncovers a hidden prophecy and battles a dark sorcerer to save his realm.',
        'A group of explorers discovers a mystical star that holds the key to the universe’s secrets.',
        'A hero confronts an ancient sorcerer threatening to plunge the world into darkness.',
        'A detective investigates a series of mysterious disappearances under the moon’s eerie glow.',
        'Astronauts explore a distant planet, uncovering alien technology and hidden dangers.',
        'A scientist trapped in a time loop must solve a puzzle to prevent a global catastrophe.',
        'Students at a magical academy face challenges and rivalries while mastering their powers.',
        'A lone wanderer leads a tribe through a harsh desert to find a fabled oasis.',
        'Hackers lead a rebellion against a dystopian regime controlling the world’s data.',
        'An adventurer seeks a lost kingdom buried beneath the sands of time.',
        'A poet hears voices from the past, guiding her to uncover a forgotten civilization.',
        'Pilots in a futuristic air race uncover a conspiracy threatening global peace.',
        'A cunning thief pulls off heists while evading a relentless detective.',
        'A marine biologist discovers a hidden underwater city with ancient secrets.',
        'A firefighter with a troubled past finds redemption battling a massive inferno.',
        'A crew on a quantum-powered ship navigates uncharted dimensions to return home.',
        'A child enters a magical forest where mythical creatures guard a powerful artifact.',
        'A psychologist helps a patient confront nightmares that turn out to be real threats.',
        'The last of an ancient order protects a sacred relic from a ruthless warlord.',
        'A new star appears in the sky, triggering events that could reshape the cosmos.'
    ]
}

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('movies.csv', index=False)
print("Dataset created and saved as 'movies.csv'")

Dataset created and saved as 'movies.csv'


In [13]:
# Install required packages
!pip install streamlit pyngrok nltk pandas numpy scikit-learn

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from pyngrok import ngrok
import subprocess
import time

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Load dataset from CSV
@st.cache_data
def load_data():
    try:
        df = pd.read_csv('movies.csv')
        if 'Movie_Name' not in df.columns or 'Storyline' not in df.columns:
            raise ValueError("CSV must contain 'Movie_Name' and 'Storyline' columns")
        df['Processed_Storyline'] = df['Storyline'].apply(preprocess_text)
        return df
    except FileNotFoundError:
        st.error("movies.csv not found. Please upload the CSV file.")
        return None
    except Exception as e:
        st.error(f"Error loading CSV: {e}")
        return None

# Write Streamlit app to file
with open('app.py', 'w') as f:
    f.write('''
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

@st.cache_data
def load_data():
    try:
        df = pd.read_csv('movies.csv')
        if 'Movie_Name' not in df.columns or 'Storyline' not in df.columns:
            raise ValueError("CSV must contain 'Movie_Name' and 'Storyline' columns")
        df['Processed_Storyline'] = df['Storyline'].apply(preprocess_text)
        return df
    except FileNotFoundError:
        st.error("movies.csv not found. Please upload the CSV file.")
        return None
    except Exception as e:
        st.error(f"Error loading CSV: {e}")
        return None

st.title('Movie Recommendation System')
st.write('Enter a movie storyline to get recommendations')

df = load_data()
if df is not None:
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['Processed_Storyline'])

    user_input = st.text_area('Enter a movie storyline:',
                            'A young wizard begins his journey at a magical school where he makes friends and enemies, facing dark forces along the way')

    if st.button('Get Recommendations'):
        processed_input = preprocess_text(user_input)
        user_tfidf = tfidf.transform([processed_input])
        cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix)
        sim_scores = list(enumerate(cosine_sim[0]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_5_indices = [i[0] for i in sim_scores[:5]]

        st.subheader('Top 5 Recommended Movies:')
        for idx in top_5_indices:
            st.write(f"**{df['Movie_Name'].iloc[idx]}**")
            st.write(f"Storyline: {df['Storyline'].iloc[idx]}")
            st.write("---")
    ''')

# Kill existing ngrok and Streamlit processes
!pkill ngrok
!pkill streamlit

# Set ngrok auth token
try:
    !ngrok authtoken 2wVNj1AoLAIlz9SqQ6Zg6lQ4iks_469cNo7ocrA6Ny2CftnQx # Replace with your actual token
    print("ngrok auth token set successfully")
except Exception as e:
    print(f"Failed to set ngrok auth token: {e}")
    raise

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8501)
    print(f"ngrok tunnel started at: {public_url}")
except Exception as e:
    print(f"Failed to start ngrok: {e}")
    raise

# Start Streamlit in the background
try:
    process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port=8501'])
    print("Streamlit server started. Waiting for it to be ready...")
except Exception as e:
    print(f"Failed to start Streamlit: {e}")
    raise

# Wait for Streamlit to start
time.sleep(10)  # Increased to 10 seconds for stability

# Check if Streamlit is running
try:
    result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
    if 'streamlit' in result.stdout:
        print("Streamlit is running. Access it at the ngrok URL above.")
    else:
        print("Streamlit failed to start. Check for errors in the Streamlit output.")
except Exception as e:
    print(f"Error checking Streamlit process: {e}")

# Check if movies.csv exists
try:
    with open('movies.csv', 'r') as f:
        print("movies.csv found in the current directory.")
except FileNotFoundError:
    print("movies.csv not found. Please run the dataset creation code first.")

# Keep the cell running to maintain the tunnel
try:
    input("Press Enter to stop the server and tunnel...")
except KeyboardInterrupt:
    print("Stopping server and tunnel...")
finally:
    ngrok.kill()
    process.terminate()
    print("Server and tunnel stopped.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-05-01 18:17:34.374 No runtime found, using MemoryCacheStorageManager


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ngrok auth token set successfully
ngrok tunnel started at: NgrokTunnel: "https://f6b4-34-106-40-163.ngrok-free.app" -> "http://localhost:8501"
Streamlit server started. Waiting for it to be ready...
Streamlit is running. Access it at the ngrok URL above.
movies.csv found in the current directory.
Press Enter to stop the server and tunnel...
Server and tunnel stopped.
